In [46]:
import numpy as np
from scipy.spatial import Delaunay, minkowski_distance
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.sparse import coo_array
import scipy.sparse.linalg as spla
import pygmsh
import meshio
from icosphere import icosphere

points = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 0]])
point_normals = np.array([[0, 0, 1]]*4)
simplices = [[2,3,0],[3,1,0]]


points = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0.7, 0], [0.7, 1, 0]])
point_normals = np.array([[0, 0, 1]]*5)
simplices = [[2,3,0],[3,4,1,0]]


points = np.array([[0. ,        0.,         0.        ],
 [0.,         1.,         0.        ],
 [1.,         0.,         0.        ],
 [1.,         1.,         0.        ],
 [0.430433,   0.35267956, 0.        ],
 [0.54382166, 0.63044885, 0.        ],
 [0.48164577, 0.83756961, 0.        ],
 [0.85101947, 0.7123014,  0.        ],
 [0.10183893, 0.30035364, 0.        ],
 [0.27944051, 0.68306335, 0.        ]])
point_normals = np.array([[0, 0, 1]]*10)
simplices =  Delaunay(points[:,:-1]).simplices

'''
nseg, points, point_normals = 5, [], []
for i in range(nseg+1):
    for j in range(nseg+1):
        points.append([i/nseg,j/nseg,0])
        point_normals.append([0,0,1])
points = np.array(points)
point_normals = np.array(point_normals)
simplices =  Delaunay(points[:,:-1]).simplices

with pygmsh.geo.Geometry() as geom:
    geom.add_circle([0.0, 0.0], 1.0, mesh_size=0.2)
    mesh = geom.generate_mesh()

points = mesh.points
point_normals = np.array([[0, 0, 1]]*(mesh.points.shape[0]))
simplices = mesh.cells_dict['triangle']
'''

points, simplices = icosphere(30)
point_normals = points / np.linalg.norm(points,axis=-1,keepdims=True)

barycenters = np.array([np.mean(points[simplice],axis=0) for simplice in simplices])
barynormals = np.array([np.mean(point_normals[simplice],axis=0) for simplice in simplices])

In [47]:
# Generate edge list and edge owner & neighbor list
edges = {}
for simplex_index, simplex in enumerate(simplices):
    for i in range(len(simplex)):
        # Create sorted tuple to identify edges uniquely
        edge = tuple(sorted((simplex[i], simplex[(i + 1) % len(simplex)])))
        if edge not in edges:
            edges[edge] = []
        edges[edge].append(simplex_index)

edges_to_vertices = []
owners = []
neighbours = []
for edge, _simplices in edges.items():
    edges_to_vertices.append(edge)
    owners.append(_simplices[0])
    neighbours.append(-1 if len(_simplices) == 1 else _simplices[1] )

# Displaying the owner and neighbor list
#print("Edge, Owner and Neighbor List:")
#for vertices_per_face, owner, neighbour in zip(edges_to_vertices, owners, neighbours):
#    print(f"Edge {vertices_per_face} has owner {owner} and neighbor {neighbour}")

#print(neighbours)


In [48]:
# Compute edge length (face area), edge (face) center and centroid of edges (faces)
edge_lengths = minkowski_distance(points[[x[0] for x in edges_to_vertices]],points[[x[1] for x in edges_to_vertices]])
#print(edge_lengths)

edge_centers = np.array([np.mean(np.array([points[v0] for v0 in v]),axis=0) for v in edges_to_vertices])
#print(edge_centers)

# Compute edge tangent, co-tangent and normals
edge_tangents = np.array([(points[v[1]] - points[v[0]])/l for v, l in zip(edges_to_vertices, edge_lengths)])
#print(edge_tangents)

edge_bitangents = np.array([np.mean(np.array([point_normals[v0] for v0 in v]),axis=0) for v in edges_to_vertices])
#print(edge_bitangents)

edge_normals = np.array([np.cross(t, r) for t, r in zip(edge_tangents, edge_bitangents)])
#print(edge_normals)

def herons_formula(a, b, c):
    """ Calculate the area of a triangle given the lengths of its sides using Heron's formula. """
    s = (a + b + c) / 2
    return np.sqrt(s * (s - a) * (s - b) * (s - c))

def area_of_polygon(points, vertices):
    """ Calculate the area of a polygon by triangulating it into multiple triangles. """
    if len(vertices) == 3:  # It's a triangle
        a = minkowski_distance(points[vertices[0]], points[vertices[1]])
        b = minkowski_distance(points[vertices[1]], points[vertices[2]])
        c = minkowski_distance(points[vertices[2]], points[vertices[0]])
        return herons_formula(a, b, c)
    else:  # It's a polygon with more than 3 vertices, triangulate it
        area = 0
        for i in range(1, len(vertices) - 1):
            a = minkowski_distance(points[vertices[0]], points[vertices[i]])
            b = minkowski_distance(points[vertices[i]], points[vertices[i + 1]])
            c = minkowski_distance(points[vertices[i + 1]], points[vertices[0]])
            area += herons_formula(a, b, c)
        return area

# Calculate the area for each simplex
areas = [area_of_polygon(points, simplex) for simplex in simplices]
#print("Areas of the polygons:", areas)

In [49]:
# Edge (Face) weighing factor
edge_weighing_factor = []

def point_to_line_distance(point, x0, x1):
    """ Calculate the normal distance from a point to a line defined by two points (x0 and x1) in 3D,
        handling edge cases such as degenerate lines. """
    # Convert points to numpy arrays if not already
    point = np.asarray(point)
    x0 = np.asarray(x0)
    x1 = np.asarray(x1)

    # Calculate the direction vector of the line
    d = x1 - x0

    # Check if the direction vector is zero (degenerate line segment)
    if np.allclose(d, 0):
        print("Warning: The input points for the line are the same. The line is degenerate.")
        return np.linalg.norm(point - x0)

    # Calculate the vector from x0 to the point
    p = point - x0

    # Calculate the cross product of d and p
    c = np.cross(d, p)

    # Calculate the distance from the point to the line (magnitude of cross product divided by magnitude of d)
    distance = np.linalg.norm(c) / np.linalg.norm(d)
    
    return distance

for owner, neighbour, vertices in zip(owners, neighbours, edges_to_vertices):
    distance_to_owner = point_to_line_distance(barycenters[owner], points[vertices[0]], points[vertices[1]])
    distance_to_neighbour = point_to_line_distance(barycenters[neighbour], points[vertices[0]], points[vertices[1]]) \
        if neighbour != -1 else 0
    edge_weighing_factor.append(distance_to_owner / (distance_to_owner + distance_to_neighbour))

edge_weighing_factor = np.array(edge_weighing_factor)
#print(edge_weighing_factor)

# Skewness
def project_point_onto_plane(v, p, n):
    n = np.array(n)
    v = np.array(v)
    p = np.array(p)
    return v - np.dot(v - p, n) / np.dot(n, n) * n

def find_intersection(v1, v2, p, t, n):
    v1_proj = project_point_onto_plane(v1, p, n)
    v2_proj = project_point_onto_plane(v2, p, n)
    d1 = v2_proj - v1_proj
    t = np.array(t)

    # Matrix to solve for parameters t (for the projected line) and s (for the line in the plane)
    A = np.column_stack([d1, -t])
    b = np.array(p) - v1_proj

    # Check if the matrix A is singular (i.e., lines are parallel)
    if np.linalg.matrix_rank(A) < 2:
        return None  # Lines are parallel or coincident; no unique intersection

    # Solve the linear system
    params, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    if residuals.size > 0 and not np.isclose(residuals, 0):
        return None  # The lines do not intersect

    intersection_point = v1_proj + params[0] * d1
    return intersection_point

skewness = []
skewness_vector = []
for owner, neighbour, center, length, tangent, bitangent in zip(owners, neighbours, edge_centers, edge_lengths, edge_tangents, edge_bitangents):
    intersection = find_intersection(barycenters[owner], barycenters[neighbour], center, tangent, bitangent) if neighbour != -1 else None
    distance = minkowski_distance(intersection, center) if neighbour != -1 else 0
    skewness.append(2.0 * distance / length)
    skewness_vector.append(center - intersection if neighbour != -1 else center - center)

skewness = np.array(skewness)
skewness_vector = np.array(skewness_vector)
#print(skewness, skewness_vector)

# Compute surface gradient $$ (\nabla \phi)_f = \frac{1}{cos(\theta)} \frac{\phi_F - \phi_C}{d_{CF}} S_f + (\nabla \phi)_f \cdot (\bm{n} - \frac{\bm{e}}{cos(\theta)}) S_f  $$

In [50]:
# \phi = x^2 + y^2 + (xy)^2
xc = barycenters[:,0]
yc = barycenters[:,1]
zc = barycenters[:,2]
phi = 0.01*(xc**2 + yc**2 + (xc*yc)**2)

xf = edge_centers[:,0]
yf = edge_centers[:,1]
zf = edge_centers[:,1]
phi_f_exact = xf**2 + yf**2 + (xf*yf)**2
#print(phi_f_exact)

dphi_f_exact = np.array([[2*x + 2*x*y**2, 2*y+2*y*x**2, 0] for x, y in zip(xf, yf)])
#print(dphi_f_exact)

# Init dphi_f
dphi_f = np.zeros_like(edge_centers)

for _ in range(2):
    # Estimate phi_f
    phi_f0 = edge_weighing_factor * phi[owners] + np.where(neighbours != -1,(1.0 - edge_weighing_factor) * phi[neighbours],np.zeros_like(owners))

    # Correct phi_f
    phi_f = phi_f0 + np.einsum('ij,ij->i', dphi_f, skewness_vector)

    # Estimate \nabla \phi_C
    dphi = np.zeros((len(areas),3))
    for phi_fi, normal, center, sf, owner, neighbour in zip(phi_f, edge_normals, edge_centers, edge_lengths, owners, neighbours):
        e = center - barycenters[owner] # (todo) need revision
        direction = 1 if np.dot(e, normal) > 0 else -1
        dphi[owner] += phi_fi * sf * normal / areas[owner] * direction
        if neighbour != -1:
            dphi[neighbour] -= phi_fi * sf * normal / areas[neighbour] * direction
            
    # (here to enforce boundary condition)

    # Estimate phi_f
    dphi_f = edge_weighing_factor[:, np.newaxis] * dphi[owners] + np.where(neighbours != -1, (1.0 - edge_weighing_factor[:, np.newaxis]) * dphi[neighbours],np.zeros_like(dphi[owners]))
    
    # Enforce neumann boundary
    for edge_ind, center, neighbour in zip(range(len(neighbours)), edge_centers, neighbours):
        if neighbour == -1:
            dphi_f[edge_ind] = 0.0
            
    print(np.linalg.norm(dphi_f - dphi_f_exact))
#print(dphi_f)

# Construct diffusion operator
dt = 1
ref_cell = 0
row  = []
col  = []
data = []
b = np.zeros((len(areas),))
for dphi_fi, normal, center, sf, owner, neighbour in zip(dphi_f, edge_normals, edge_centers, edge_lengths, owners, neighbours):
        direction = 1 if np.dot(center - barycenters[owner], normal) > 0 else -1
        if neighbour != -1:
            cf = barycenters[neighbour] - barycenters[owner]
            d_cf = np.linalg.norm(cf)
            e = cf/d_cf
            ef = sf/np.dot(e, normal*direction)
            row.append(owner)
            col.append(owner)
            data.append(dt*ef/d_cf)
            row.append(owner)
            col.append(neighbour)
            data.append(-dt*ef/d_cf)
            row.append(neighbour)
            col.append(neighbour)
            data.append(dt*ef/d_cf)
            row.append(neighbour)
            col.append(owner)
            data.append(-dt*ef/d_cf)
            b[owner] += dt*np.dot(dphi_fi, normal*direction-e/np.dot(e, normal*direction)) * sf
            b[neighbour] -= dt*np.dot(dphi_fi, normal*direction-e/np.dot(e, normal*direction)) * sf

# Time derivative
#for cell_idx in range(len(areas)):
#    row.append(cell_idx)
#    col.append(cell_idx)
#    data.append(1.0)
#    b[cell_idx] += phi[cell_idx]

#for cell_idx in range(len(areas)):
#    b[cell_idx] += xc[cell_idx] * areas[cell_idx]
Jf = coo_array((data, (row, col)), shape=(len(areas), len(areas)))

# Solve Ax = b using Conjugate Gradient method
phi_new, info = spla.cg(Jf, b)

# Check if the solution was successful
if info == 0:
    print("Solution found!")
    #print(phi_new)
else:
    print("Conjugate Gradient did not converge. Info:", info)

Jf = Jf.todense()
#print(Jf, b)

328.8446193431304
328.84462244714763
Solution found!


In [51]:
mesh = meshio.Mesh(
    points,
    [("triangle", simplices),],
    # Optionally provide extra data on points, cells, etc.
    #point_data={"T": [0.3, -1.2, 0.5, 0.7, 0.0, -3.0]},
    # Each item in cell data must match the cells array
    cell_data={"phi": [phi_new]},
)
mesh.write(
    "icosphere_residual_test.vtk",  # str, os.PathLike, or buffer/open file
    # file_format="vtk",  # optional if first argument is a path; inferred from extension
)

In [52]:
'''
# Setup for a 3D plot
fig = plt.figure(figsize=(22,8))
ax = fig.add_subplot(111, projection='3d')

# Plot each simplex
for simplex in simplices:
    polygon = points[simplex]
    ax.add_collection3d(Poly3DCollection([polygon], facecolors='grey', linewidths=.2, edgecolors='k', alpha=.75))

# Set plot display parameters
#ax.scatter(points[:,0], points[:,1], points[:,2], color='b')  # Plot the points
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Adjusting the scale for better visualization
max_range = np.array([points[:,0].max()-points[:,0].min(), 
                      points[:,1].max()-points[:,1].min(), 
                      points[:,2].max()-points[:,2].min()]).max() / 2.0
mid_x = (points[:,0].max()+points[:,0].min()) * 0.5
mid_y = (points[:,1].max()+points[:,1].min()) * 0.5
mid_z = (points[:,2].max()+points[:,2].min()) * 0.5
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

#ax.scatter(barycenters[:,0], barycenters[:,1], barycenters[:,2], color='r')  # Plot the points
#ax.scatter(edge_centers[:,0], edge_centers[:,1], edge_centers[:,2], color='g')  # Plot the points
#ax.quiver(xf, yf, 0*xf, dphi_f_exact[:,0], dphi_f_exact[:,1], 0*xf, color='m',length=0.1)
#ax.quiver(xf, yf, 0*xf, dphi_f[:,0], dphi_f[:,1], 0*xf, color='g',length=0.1)
#ax.quiver(xc, yc, 0*xc, dphi[:,0], dphi[:,1], 0*xc, color='g',length=0.1)
#ax.quiver(xf, yf, 0*xf, 0*xf, 0*xf, phi_f_exact, color='m',length=0.25)
#ax.quiver(xf, yf, 0*xf, 0*xf, 0*xf, phi_f0, color='b',length=0.25)
#ax.quiver(xf, yf, 0*xf, 0*xf, 0*xf, phi_f, color='g',length=0.25)
#ax.quiver(xc, yc, zc, phi*barynormals[:,0], phi*barynormals[:,1], phi*barynormals[:,2], color='m',length=0.25)
ax.quiver(xc, yc, zc, phi_new*barynormals[:,0], phi_new*barynormals[:,1], phi_new*barynormals[:,2], color='g',length=0.25)
plt.show()
'''

"\n# Setup for a 3D plot\nfig = plt.figure(figsize=(22,8))\nax = fig.add_subplot(111, projection='3d')\n\n# Plot each simplex\nfor simplex in simplices:\n    polygon = points[simplex]\n    ax.add_collection3d(Poly3DCollection([polygon], facecolors='grey', linewidths=.2, edgecolors='k', alpha=.75))\n\n# Set plot display parameters\n#ax.scatter(points[:,0], points[:,1], points[:,2], color='b')  # Plot the points\nax.set_xlabel('X')\nax.set_ylabel('Y')\nax.set_zlabel('Z')\n\n# Adjusting the scale for better visualization\nmax_range = np.array([points[:,0].max()-points[:,0].min(), \n                      points[:,1].max()-points[:,1].min(), \n                      points[:,2].max()-points[:,2].min()]).max() / 2.0\nmid_x = (points[:,0].max()+points[:,0].min()) * 0.5\nmid_y = (points[:,1].max()+points[:,1].min()) * 0.5\nmid_z = (points[:,2].max()+points[:,2].min()) * 0.5\nax.set_xlim(mid_x - max_range, mid_x + max_range)\nax.set_ylim(mid_y - max_range, mid_y + max_range)\nax.set_zlim(mid_z -